In [1]:
input_file_path = 'redcard.csv'

## This exploration was done by Sean Markert, for the redCard data set. The purpose of the challenge is to check whether or not there is bias against darker skin players, and they recieve more red cards.


- ### The table we have been provided looks at red cards awarded by refs to players. Each row is based off of a ref-player pair.

In [2]:
import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline
plt.style.use( 'dark_background' )
plt.rcParams[ 'figure.figsize' ] = ( 14, 10 )

From the paper link:

playerShort 
- short player ID\

player 
- player name\

club
- player club\

leagueCountry
- country of player club (England, Germany, France, and Spain)\

birthday
- player birthday\

height
- player height (in cm)\

weight
- player weight (in kg)\

position
- detailed player position \

games
- number of games in the player-referee dyad\

victories
- victories in the player-referee dyad\

ties
- ties in the player-referee dyad\

defeats
- losses in the player-referee dyad\

goals
- goals scored by a player in the player-referee dyad\

yellowCards
- number of yellow cards player received from referee\

yellowReds
- number of yellow-red cards player received from referee\

redCards
- number of red cards player received from referee\

photoID
- ID of player photo (if available)\

rater1
- skin rating of photo by rater 1 (5-point scale ranging from \'93very light skin\'94 to '93very dark skin\'94)\

rater2
- skin rating of photo by rater 2 (5-point scale ranging from \'93very light skin\'94 to \'93very dark skin\'94)\

refNum
- unique referee ID number (referee name removed for anonymizing purposes)\

refCountry
- unique referee country ID number (country name removed for anonymizing purposes)\

meanIAT
- mean implicit bias score (using the race IAT) for referee country, higher values correspond to faster white | good, black | bad associations \

nIAT
- sample size for race IAT in that particular country\

seIAT
- standard error for mean estimate of race IAT       \

meanExp
- mean explicit bias score (using a racial thermometer task) for referee country, higher values correspond to greater feelings of warmth toward whites versus blacks\

nExp
- sample size for explicit bias in that particular country\

seExp
- standard error for mean estimate of explicit bias measure\


The biggest thing we are going to want to look at to answer the question is:
- ### yellowCards
- ### yellowReds
- ### redCards
- ### rater1
- ### rater2

Things that will have a strong effect on this result will be:
- refNum
- refCountry
- meanIAT
- nIAT
- nIAT
- seIAT
- meanExp
- nExp
- seExp

Let's read in the data and have a look at it

In [3]:
raw_df = pd.read_csv( input_file_path )

print( raw_df.shape )
print( raw_df.head() )

(146028, 28)
     playerShort         player             club leagueCountry    birthday  \
0  lucas-wilchez  Lucas Wilchez    Real Zaragoza         Spain  31.08.1983   
1     john-utaka     John Utaka  Montpellier HSC        France  08.01.1982   
2    abdon-prats    Abdón Prats     RCD Mallorca         Spain  17.12.1992   
3     pablo-mari     Pablo Marí     RCD Mallorca         Spain  31.08.1993   
4     ruben-pena     Rubén Peña  Real Valladolid         Spain  18.07.1991   

   height  weight              position  games  victories    ...     rater2  \
0   177.0    72.0  Attacking Midfielder      1          0    ...       0.50   
1   179.0    82.0          Right Winger      1          0    ...       0.75   
2   181.0    79.0                   NaN      1          0    ...        NaN   
3   191.0    87.0           Center Back      1          1    ...        NaN   
4   172.0    70.0      Right Midfielder      1          1    ...        NaN   

   refNum  refCountry  Alpha_3   meanIAT   

That's a lot of players, and a fair amount of features

In [4]:
raw_df.columns.values

array(['playerShort', 'player', 'club', 'leagueCountry', 'birthday',
       'height', 'weight', 'position', 'games', 'victories', 'ties',
       'defeats', 'goals', 'yellowCards', 'yellowReds', 'redCards',
       'photoID', 'rater1', 'rater2', 'refNum', 'refCountry', 'Alpha_3',
       'meanIAT', 'nIAT', 'seIAT', 'meanExp', 'nExp', 'seExp'],
      dtype=object)

Now let's check for sparsity

In [7]:
raw_df.isnull().sum()

playerShort          0
player               0
club                 0
leagueCountry        0
birthday             0
height             263
weight            2243
position         17726
games                0
victories            0
ties                 0
defeats              0
goals                0
yellowCards          0
yellowReds           0
redCards             0
photoID          21407
rater1           21407
rater2           21407
refNum               0
refCountry           0
Alpha_3              1
meanIAT            163
nIAT               163
seIAT              163
meanExp            163
nExp               163
seExp              163
dtype: int64

So we are missing some heights, weights, and positions. That shouldn't be too important for the question we were posed. 

The rater data is important for this question, so we will need to fill this data. If there are duplicate players, we can possibly fill these values, or at least use group by's to handle this.

We are also missing some of the implicit bias normalization info. This may be an issue, but again may be able to handle this if there are duplicates in the data.

Let's see if the player id's are unique

In [6]:
print( raw_df['playerShort'].unique().shape[0] == raw_df.shape[0] )
print( raw_df['playerShort'].unique().shape[0]  )
print( raw_df.shape[0] )

False
2053
146028


These are not unique, but likely tied to a player's name. Let's check what kind of numbers we are working with by player.

In [13]:
player_group = raw_df.groupby( 'playerShort' ).size()

print( player_group.head(10) )
print( player_group.sum() == raw_df.shape[0] )

playerShort
aaron-hughes              166
aaron-hunt                 99
aaron-lennon              101
aaron-ramsey              104
abdelhamid-el-kaoutari     37
abdon-prats                33
abdou-dampha               18
abdou-traore_2             42
abdoul-camara              67
abdoulaye-diallo_2         20
dtype: int64
True


Alright, let's do the same for refs.

In [14]:
ref_group = raw_df.groupby( 'refNum' ).size()

print( raw_df['refNum'].unique().shape[0] )
print( ref_group.head(10) )

3147
refNum
1       1
2       1
3       3
4      30
5       1
6       9
7     178
8       1
9       3
10      1
dtype: int64


Some refs only have 1 game! This may make some of the calibrating for biases difficult if there is missing data.

Let's move to visualizing some of these.